In [55]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import random

PEGASOS = False
KERNEL = False
OPT_STEP = 1e-12 # best step size we found
OPT_C = 5000 # best c value we found
OPT_LAMBDA = 1e-5
EPOCHS = 10

dev = True
step_sizes = [-14, -13, -12, -11, -10]
#step_sizes = [1e-14, 1e-13, 1e-12, 1e-11, 1e-10]
C_vals = [1000, 4000, 7000, 10000, 13000, 16000]
lambda_vals = [1e-1, 1e-3, 1e-5, 1e-7, 1e-9]
sigma = 10
m = 1000

test_errors = []

def main():                                                                   
    df_train = pd.read_csv('../data/mnist_train.csv')                         
    df_train['bias'] = 1                                                      
                                                                              
    if dev:                                                                   
        if PEGASOS:                                                           
            pegasos_cross_validation(df_train)                                
        else:                                                                 
            sgd_cross_validation(df_train)                                    
    else:                                                                     
        X = df_train.drop('label', axis=1).values                             
        if KERNEL:                                                            
            X = transform(X)                                                  
        y = df_train['label'].values                                          
                                                                              
        df_test = pd.read_csv('../data/mnist_test.csv')                       
        df_test['bias'] = 1                                                   
        X_test = df_test.drop('label', axis=1).values                         
        if KERNEL:                                                            
            X_test = transform(X_test)                                        
        y_test = df_test['label'].values                                      
                                                                              
        w_s = {}  # a map from classification {0, ..., 9} to weight vector    
        for class_val in range(10):                                           
            if PEGASOS:                                                       
                w = pegasos(X, y, class_val, OPT_LAMBDA, EPOCHS)              
            else:                                                             
                w = sgd(X, y, class_val, OPT_STEP, OPT_C, EPOCHS)             
            w_s[class_val] = w                                                
                                                                              
        test_error = calculate_test_error(w_s, X_test, y_test)                
        print('test error: ', test_error)                                     


def pegasos_cross_validation(df_train):
    test_errors = []
    for lambda_val in lambda_vals:
        print('lambda: ', lambda_val)

        df_train_dev, df_dev = np.split(df_train.sample(frac=1), [int(.8 * len(df_train))])
        X = df_train_dev.drop('label', axis=1).values
        y = df_train_dev['label'].values
        X_dev = df_dev.drop('label', axis=1).values
        y_dev = df_dev['label'].values

        # Training the SVM classifiers
        w_s = {}  # a map from classification {0, ..., 9} to weight vector
        for class_val in range(10):
            w = pegasos(X, y, class_val, lambda_val, EPOCHS)
            w_s[class_val] = w

        test_error = calculate_test_error(w_s, X_dev, y_dev)
        test_errors.append(test_error)
        print('test error: ', test_error)

    plt.plot(lambda_vals, test_errors)
    plt.title('PEGASOS Lambda vs. Validation Error')
    plt.xscale('log')
    plt.xlabel('lambda')
    plt.ylabel('% misclassified')
    plt.savefig('pegasos_lambda.png')


def sgd_cross_validation(df_train):
    test_errors = []
    for step_size in step_sizes:
        for C in C_vals:
            print('step size: ', step_size, ' c: ', C)

            df_train_dev, df_dev = np.split(df_train.sample(frac=1), [int(.8 * len(df_train))])
            X = df_train_dev.drop('label', axis=1).values
            y = df_train_dev['label'].values
            X_dev = df_dev.drop("label", axis=1).values
            y_dev = df_dev['label'].values

            # Training the SVM classifiers
            w_s = {} # a map from classification {0, ..., 9} to weight vector
            for class_val in range(10):
                if PEGASOS:
                    w = pegasos(X, y, class_val, EPOCHS)
                else:
                    w = sgd(X, y, class_val, step_size, C, EPOCHS)
                w_s[class_val] = w

            test_error = calculate_test_error(w_s, X_dev, y_dev)
            test_errors.append(test_error)
            print('test error: ', test_error)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')


# transforms X into m-dimensional feature vectors using RFF and RBF kernel
# Make sure this function works for both 1D and 2D NumPy arrays.
def transform(X):
    #np.random.seed(0)
    b = np.random.rand(m) * 2 * np.pi
    w = np.random.multivariate_normal(np.zeros(X.shape[1]), sigma**2 * np.identity(X.shape[1]), m)
    transformed = (2.0 / m)**0.5 * np.cos(np.dot(X, w.T) + b)
    # feature normalization
    transformed = (transformed - np.mean(transformed, 0)) / np.std(transformed, 0)

    return transformed


def sgd(X, y, class_val, step_size, C, epochs):
    w = np.zeros(len(X[0]))
    for epoch in range(epochs):
        points = list(range(len(X)))
        random.shuffle(points)
        for point in points:
            # is this point in the class we are looking for?
            if y[point] == class_val:
                y_i = 1
            else:
                y_i = -1
            # update step
            w += step_size * (C * max(0, 1 - y_i * np.dot(X[point], w)) * (y_i * X[point]) - 2 * w)
    return w


def pegasos(X, y, class_val, lambda_val, epochs):
    w = np.zeros(X.shape[1])
    t = 1

    for epoch in range(epochs):
        points = list(range(X.shape[0]))
        random.shuffle(points)
        for point in points:
            if y[point] == class_val:
                y_i = 1
            else:
                y_i = -1

            learning_rate = 1 / (lambda_val * t)
            w = (1 - learning_rate * lambda_val) * w + learning_rate * hinge_loss_gradient(w, X[point], y_i)

            # optional projection step, currently messing things up
            #new_w = min(1, ((1 / lambda_val**0.5) / np.linalg.norm(new_w))) * new_w

            t += 1

    return w


# calculate the gradient of the hinge loss function
def hinge_loss_gradient(w, x, y):
    if np.dot(w, x) * y >= 1:
        return 0
    else:
        return y * x


def calculate_test_error(w_s, X_test, y_test):
    misclassified = 0
    for idx in range(X_test.shape[0]):
        x = X_test[idx]

        # Find class with maximum margin and classify x as that class
        # Fencepost: initialize max with margin denoted by 0
        max_margin = np.dot(w_s[0], x)
        max_class = 0
        for class_val in range(1, 10):
            margin = np.dot(w_s[class_val], x)
            if margin > max_margin:
                max_margin = margin
                max_class = class_val

        #print(max_class, ' ', y_test[idx])
        if max_class != y_test[idx]:
            misclassified += 1

    print('misclassified: ', misclassified)
    print('total: ', len(X_test))
    return misclassified / len(X_test)

In [43]:
main()

step size:  1e-14  c:  1000
misclassified:  2474
total:  12000
test error:  0.20616666666666666
step size:  1e-14  c:  4000
misclassified:  1917
total:  12000
test error:  0.15975
step size:  1e-14  c:  7000
misclassified:  1796
total:  12000
test error:  0.14966666666666667
step size:  1e-14  c:  10000
misclassified:  1700
total:  12000
test error:  0.14166666666666666
step size:  1e-14  c:  13000
misclassified:  1558
total:  12000
test error:  0.12983333333333333
step size:  1e-14  c:  16000
misclassified:  1556
total:  12000
test error:  0.12966666666666668
step size:  1e-13  c:  1000
misclassified:  1624
total:  12000
test error:  0.13533333333333333
step size:  1e-13  c:  4000
misclassified:  1352
total:  12000
test error:  0.11266666666666666
step size:  1e-13  c:  7000
misclassified:  1238
total:  12000
test error:  0.10316666666666667
step size:  1e-13  c:  10000
misclassified:  1245
total:  12000
test error:  0.10375
step size:  1e-13  c:  13000
misclassified:  1204
total:  12

/Users/dericp/.pyenv/versions/3.6.0/envs/digit-recognition/lib/python3.6/site-packages/ipykernel/__main__.py:140: RuntimeWarning: overflow encountered in multiply
/Users/dericp/.pyenv/versions/3.6.0/envs/digit-recognition/lib/python3.6/site-packages/ipykernel/__main__.py:140: RuntimeWarning: invalid value encountered in add


misclassified:  10862
total:  12000
test error:  0.9051666666666667
step size:  1e-10  c:  10000
misclassified:  10882
total:  12000
test error:  0.9068333333333334
step size:  1e-10  c:  13000
misclassified:  10850
total:  12000
test error:  0.9041666666666667
step size:  1e-10  c:  16000
misclassified:  10805
total:  12000
test error:  0.9004166666666666


In [56]:
X, Y = np.meshgrid(step_sizes, C_vals)
Z = [[0.20616666666666666, 0.15975, 0.14966666666666667, 0.14166666666666666, 0.12983333333333333], [0.12966666666666668, 0.13533333333333333, 0.11266666666666666, 0.10316666666666667, 0.10375], [0.10033333333333333, 0.09733333333333333, 0.10341666666666667, 0.09758333333333333, 0.09716666666666667], [0.09391666666666666, 0.09433333333333334, 0.097, 0.09258333333333334, 0.09833333333333333], [0.11066666666666666, 0.12, 0.13533333333333333, 0.13441666666666666, 0.10883333333333334], [0.5349166666666667, 0.9051666666666667, 0.9068333333333334, 0.9041666666666667, 0.9004166666666666]]
#Z = [[0.09325, 0.11375, 0.5650833333333334, 0.8989166666666667, 0.9], [0.09425, 0.09858333333333333, 0.10825, 0.10916666666666666, 0.15808333333333333], [0.12841666666666668, 0.10941666666666666, 0.09558333333333334, 0.09541666666666666, 0.09575], [0.19016666666666668, 0.14433333333333334, 0.11441666666666667, 0.11066666666666666, 0.09675], [0.24641666666666667, 0.2115, 0.16375, 0.13891666666666666, 0.12083333333333333]]
plt.clf()
fig = plt.figure()
ax = fig.gca(projection='3d')
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [47]:
len(test_errors)

0

In [57]:
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

In [58]:
# Customize the z axis.
ax.set_zlim(0, 1)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
#ax.set_xlim3d(1e-14, 1e-10)
#ax.set_ylim3d(0,25000)
ax.autoscale()

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig('2dshit.png')

In [24]:
step_sizes

[1e-10, 1e-11, 1e-12, 1e-13, 1e-14]